<p><a name="pan"></a></p>

# **Operaciones sobre los datos en Pandas**

Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones vectorizadas, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.). Pandas hereda gran parte de esta funcionalidad de NumPy.

Sin embargo, Pandas incluye un par de elementos útiles: para operaciones unarias como funciones de negación y trigonométricas, estas ufuncs conservarán etiquetas de índice y columna en la salida, y para operaciones binarias como suma y multiplicación, Pandas alineará automáticamente los índices. Debido a que Pandas está diseñado para funcionar con NumPy, cualquier ufunc de NumPy funcionará en objetos Pandas Series y DataFrame.

Comencemos por definir una serie y un DataFrame simples con los cuales demostrar esto:

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = [1, 4, 16, 5]

s = pd.Series(data)
s

0     1
1     4
2    16
3     5
dtype: int64

In [ ]:
data = np.random.rand(3, 3)

df = pd.DataFrame(data)
df

,0,1,2
0,0.932444,0.616257,0.051543
1,0.246039,0.652670,0.608451
2,0.085389,0.677110,0.165984


**Preservando el índice**

Si aplicamos un ufunc de NumPy en cualquiera de estos objetos, el resultado será otro objeto de Pandas con los índices preservados:


In [ ]:
np.exp(s)

0    2.718282e+00
1    5.459815e+01
2    8.886111e+06
3    1.484132e+02
dtype: float64

In [ ]:
np.log(df)

,0,1,2
0,-0.069946,-0.484091,-2.965335
1,-1.402263,-0.426684,-0.496838
2,-2.460539,-0.389921,-1.795864


**Alineación del índice**


**Series**

Para operaciones binarias con dos objetos (Series o DataFrames), Pandas alineará los índices en el proceso de realizar la operación.


In [ ]:
d_poblacion = {"Brasil":210147125, "Colombia": 50372424, "Argentina":44938712}

s_poblacion = pd.Series(d_poblacion)
s_poblacion

Brasil       210147125
Colombia      50372424
Argentina     44938712
dtype: int64

In [ ]:
d_area = {"Brasil":8514877, "Colombia":1141748, "Perú": 2792600}

s_area = pd.Series(d_area)
s_area

Brasil      8514877
Colombia    1141748
Perú        2792600
dtype: int64

Veamos qué sucede cuando dividimos estas series para calcular la densidad de población:

In [ ]:
s_poblacion / s_area

Argentina          NaN
Brasil       24.679995
Colombia     44.118688
Perú               NaN
dtype: float64

El arreglo resultante contiene la unión de los índices de los dos arreglos de entrada, que podrían determinarse utilizando la aritmética de conjuntos estándar de Python sobre estos índices:


In [ ]:
# intersección de los indices
s_poblacion.index & s_area.index

<ipython-input-13-b18fd33493fd>:2: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  s_poblacion.index & s_area.index


Index(['Brasil', 'Colombia'], dtype='object')

In [ ]:
# union de los indices
s_poblacion.index | s_area.index

<ipython-input-14-5f7dcb9bc809>:2: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  s_poblacion.index | s_area.index


Index(['Argentina', 'Brasil', 'Colombia', 'Perú'], dtype='object')

Cualquier elemento para el que una serie u otra no tenga una entrada se marca con `NaN` ("Not a Number"), que es la forma en que Pandas marca los datos faltantes.

Esta coincidencia de índices se implementa de esta manera para cualquiera de las expresiones aritméticas integradas de Python; los valores faltantes se rellenan con `NaN` de forma predeterminada

In [ ]:
A = pd.Series([2, 4, 6])
A

0    2
1    4
2    6
dtype: int64

In [ ]:
B = pd.Series([1, 3, 5], index = [ 1, 2, 3])
B

1    1
2    3
3    5
dtype: int64

In [ ]:
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

Si no queremos incluir `NaN` en la salida, podemos utilizar la ufunc directamente. Por ejemplo, llamar a `A.add(B)` es equivalente a llamar a `A + B`, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento en `A` o `B` que pueda faltar, mediante al kwarg `fill_value`:

In [ ]:
A.add(B, fill_value=1000)

0    1002.0
1       5.0
2       9.0
3    1005.0
dtype: float64

Note que este kwarg reemplaza `0` en los valores faltantes en las series, no directamente los `NaN` en la salida de la operación de suma.

**DataFrames**

Cuando se realizan operaciones en DataFrames se produce un tipo similar de alineación tanto para las columnas como para los índices:


In [ ]:
data = np.random.rand(3, 3)

a = pd.DataFrame(data, columns =  [f"col{i}" for i in range(3)])
a

,col0,col1,col2
0,0.874477,0.201639,0.558293
1,0.545616,0.619515,0.113853
2,0.410421,0.094448,0.266564


In [ ]:
data = np.random.rand(3, 3)

b = pd.DataFrame(data, columns =  [f"col{i}" for i in range(1, 4)])
b

,col1,col2,col3
0,0.122808,0.977872,0.299485
1,0.024213,0.733547,0.928272
2,0.597755,0.765777,0.513704


In [ ]:
a + b

,col0,col1,col2,col3
0,NaN,0.324447,1.536165,NaN
1,NaN,0.643728,0.847400,NaN
2,NaN,0.692203,1.032341,NaN


Observe que los índices están alineados correctamente independientemente de su orden en los dos objetos, y los índices del resultado están ordenados. Como con las series, podemos usar la ufunc asociada y utilizar el kwarg`fill_value`


In [ ]:
a.add(b, fill_value = 10)

,col0,col1,col2,col3
0,10.874477,0.324447,1.536165,10.299485
1,10.545616,0.643728,0.847400,10.928272
2,10.410421,0.692203,1.032341,10.513704


<p><a name="ope"></a></p>

# **Operaciones entre DataFrame y Series**

Al realizar operaciones entre un DataFrame y una Serie, la alineación del índice y la columna se mantiene de manera similar. Las operaciones entre un DataFrame y una Serie son similares a las operaciones entre un arreglo de NumPy bidimensional y uno unidimensional.

La resta entre un un arreglo bidimensional y una de sus filas se aplica por filas. En Pandas, la convención opera de manera similar:

In [ ]:
data = np.arange(9).reshape(3, 3)

df = pd.DataFrame(data, columns = ['A', 'B', 'C'])
df

,A,B,C
0,0,1,2
1,3,4,5
2,6,7,8


In [ ]:
data = np.arange(3)

s = pd.Series(data, index = ['A', 'B', 'C'])
s

A    0
B    1
C    2
dtype: int64

In [ ]:
df - s

,A,B,C
0,0,0,0
1,3,3,3
2,6,6,6


Si, en cambio, deseamos operar sobre las columnas, podemos especificar el eje explícitamente con el kwarg `axis`. Note que en este caso los índices de la fila deben coincidir con los índices del DataFrame, a diferencia del caso anterior en el que los índices de la Series coincidia con las columnas del DataFrame:


In [ ]:
df.subtract(s, axis = 1)

,A,B,C
0,0,0,0
1,3,3,3
2,6,6,6


In [ ]:
df.subtract(s, axis = 0)

,A,B,C
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
A,NaN,NaN,NaN
B,NaN,NaN,NaN
C,NaN,NaN,NaN


In [ ]:
s.reset_index(drop = True, inplace = True)
df.subtract(s, axis = 0)

,A,B,C
0,0,1,2
1,2,3,4
2,4,5,6


Estudiemos este tipo de operaciones utilizando un conjunto de datos real. En este caso utilizaremos el conjunto de datos `auto.csv` que contiene muestras de vehículos con una serie de características:

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/tomasate/Datos_Clases/main/Datos_1/auto.csv")
df.head()

,Unnamed: 0,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
0,0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,...,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1
1,1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,...,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1
2,2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,...,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1
3,3,2,164,audi,std,four,sedan,fwd,front,99.8,...,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1
4,4,2,164,audi,std,four,sedan,4wd,front,99.4,...,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1


Para comenzar, note que la columna `Unnamed: 0` corresponde a la información del índice, por lo que es una columna que no debemos incluir en el conjunto de datos.

Con el método `drop` podemos eliminar una columna, una fila, o una combinación de ambas (mirar documentación del método). El método no es *in place* por lo que debemos utilizar el kwarg `inplace` para hacer el cambio efectivo sobre el DataFrame

In [ ]:
df.drop(columns = ['Unnamed: 0'], inplace = True)
df.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,...,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,...,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,...,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1


Otra opción es utilizar el kwarg `index_col` de la función `read_csv` con el cual definimos cuál será el índice del DataFrame:

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/tomasate/Datos_Clases/main/Datos_1/auto.csv", index_col = 'Unnamed: 0')
df.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,...,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,...,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,...,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1


In [ ]:
df.columns

Index(['symboling', 'normalized-losses', 'make', 'aspiration', 'num-of-doors',
       'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length',
       'width', 'height', 'curb-weight', 'engine-type', 'num-of-cylinders',
       'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio',
       'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price',
       'city-L/100km', 'horsepower-binned', 'diesel', 'gas'],
      dtype='object')

Una vez eliminada esta columna, realicemos algunas operaciones útiles.

Por ejemplo, si quisieramos construir una nueva columna que nos muestre la relación *horsepower* / *price* podemos utilizar simplemente una operación vectorizada utilizando el operador asociado a la ufunc:

In [ ]:
df.horsepower / df.price

0      0.008225
1      0.006727
2      0.009333
3      0.007312
4      0.006590
         ...   
196    0.006768
197    0.008401
198    0.006237
199    0.004717
200    0.005039
Length: 201, dtype: float64

In [ ]:
df['ratio'] = df.horsepower / df.price
df.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,ratio
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1,0.008225
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1,0.006727
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,...,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1,0.009333
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,...,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1,0.007312
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,...,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1,0.006590


Para ordenar esta columna podemos utilizar el método `sort_values`. Con el kwarg `ascending` podemos controlar si el ordenamiento es ascendente o no:

In [ ]:
df.ratio.sort_values(ascending = False)

163    0.013729
134    0.013482
76     0.013266
47     0.013090
22     0.012819
         ...   
66     0.004365
65     0.004354
71     0.004053
63     0.003925
67     0.003892
Name: ratio, Length: 201, dtype: float64

Vemos que la observación con índice 163 contiene la mejor relación *horsepower* / *price*. Accedamos a esta observación, mirando por ejemplo la columna asociada a la constructora *make*:

In [ ]:
df.loc[163, 'make']

'toyota'

En muchas ocaciones vamos a querer acceder a esta información, sin tener la necesidad de realizar una visualización para obtener el índice asociado a la observación con el valor mayor. Para este fin podemos utilizar el método `idxmax`:

In [ ]:
id_max = df.ratio.idxmax()
df.loc[id_max, 'make']

'toyota'

Otra operación común que se aplica sobre los datos numéricos es el da la normalización de una variable (observe que el conjunto de datos ya contiene este tipo de datos numéricos). Podemos visualizar rápidamente una variable utilizando una operación vectorizada similar a la anterior. Hagamos esto para la variable `height`:

In [ ]:
df.height.describe()

count    201.000000
mean      53.766667
std        2.447822
min       47.800000
25%       52.000000
50%       54.100000
75%       55.500000
max       59.800000
Name: height, dtype: float64

In [ ]:
df['height'] = df.height / df.height.max()

In [ ]:
df.height.describe()

count    201.000000
mean       0.899108
std        0.040933
min        0.799331
25%        0.869565
50%        0.904682
75%        0.928094
max        1.000000
Name: height, dtype: float64

Otro tipo de operación común es la de asignar un valor numérico a una variable categórica de tipo `str`. Por ejemplo, si quisieramos modificar la variable `num-of-doors`



In [ ]:
df['num-of-doors'].value_counts()

four    115
two      86
Name: num-of-doors, dtype: int64

Pasando de tener los valores "four" y "two" a 4 y 2, respectivamente, podemos utilizar la función nativa de Python `map(f, a)`. Esta toma una lista o arreglo `a` y le aplica alguna función `f` a cada uno de los elementos de `a` (similar a lo que hacen las ufuncs de NumPy).

Veamos un ejemplo: creemos el arreglo `[1, 2, 3, 4]` y apliquemos `map` pasando como argumento una función que devuelva el cuadrado de los elementos:

In [ ]:
lista = [1, 2, 3, 4]

def f(x):
  return x ** 2

list(map(f, lista))

[1, 4, 9, 16]

Note que `map` no devuelve una lista sino un generador, por lo que utilizamos el constructor `list` para obtener una lista.

También podemos utilizar esta función para definir un mapeo de una variable a otra, mediante el uso de un diccionario, donde las claves representarán el valor que contiene el arreglo, y las claves los nuevos valores que les queremos dar.

En el caso de una serie, podemos utilizar `map` como un método del objeto



In [ ]:
df['num-of-doors'].map({'four' : 4, 'two': 2})

0      2
1      2
2      2
3      4
4      4
      ..
196    4
197    4
198    4
199    4
200    4
Name: num-of-doors, Length: 201, dtype: int64

Modifiquemos la variable en nuestro conjunto de datos:

In [ ]:
df['num-of-doors'] = df['num-of-doors'].map({'four' : 4, 'two': 2})

In [ ]:
df.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,ratio
0,3,122,alfa-romero,std,2,convertible,rwd,front,88.6,0.811148,...,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1,0.008225
1,3,122,alfa-romero,std,2,convertible,rwd,front,88.6,0.811148,...,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1,0.006727
2,1,122,alfa-romero,std,2,hatchback,rwd,front,94.5,0.822681,...,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1,0.009333
3,2,164,audi,std,4,sedan,fwd,front,99.8,0.848630,...,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1,0.007312
4,2,164,audi,std,4,sedan,4wd,front,99.4,0.848630,...,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1,0.006590


Más adelante veremos formas alternativas para realizar este tipo de operaciones de manera que las podamos incluir en un proceso más general.

<p><a name="agg"></a></p>

# **Agregaciones y agrupamiento**

Una parte esencial del análisis de grandes cantidades de datos es el cálculo de agregaciones como la suma, media, mediana, etc; que nos proporcionan un resumen estadístico de los datos. En esta sección, exploraremos agregaciones en Pandas.

En la sesión anterior vimos cómo aplicar funciones de agregación simples sobre los objetos de pandas, similares a las que hemos visto en los arreglos de NumPy. Veamos ahora algunas funcionalidades más complejas:



**Aggregate (agg)**:

Este método permite tener mucha más flexibilidad en las operaciones de agregación. Puede tomar una cadena de caracteres, una función o una lista de las mismas y calcular todos las agregaciones a la vez. Veamos algunos ejemplos:

In [ ]:
# aplicando agg sobre una serie numerica
# para obtener la media, el maximo
# y el minimo

df.price.agg(['mean', max, np.min])


mean    13207.129353
max     45400.000000
amin     5118.000000
Name: price, dtype: float64

In [ ]:
# aplicando agg sobre una serie categorica
# para obtener el numero de elemntos
# y el numero de valores unicos
df.make.agg(['count', 'nunique'])

count      201
nunique     22
Name: make, dtype: int64

También lo podemos aplicar sobre el dataframe completo y utilizar un diccionario para mapear qué funciones (valores) queremos aplicar a una columna particular (claves)

In [ ]:
df.agg({'price': ['mean', max, np.min, 'count'],
        'make': ['count', 'nunique']})

,price,make
mean,13207.129353,NaN
max,45400.000000,NaN
amin,5118.000000,NaN
count,201.000000,201.0
nunique,NaN,22.0


**Apply**

Este método nos permite aplicar una función arbitraria. La función debe tomar un DataFrame y devolver un objeto de Pandas o un número.

Por ejemplo, supongamos que queremos pasar de tener la variable continua *price* a una variable categórica que tome los valores *inf* y *sup* de acuerdo a un criterio particular. Podemos utilizar la media como criterio de separación:

In [ ]:
df.price.describe()

count      201.000000
mean     13207.129353
std       7947.066342
min       5118.000000
25%       7775.000000
50%      10295.000000
75%      16500.000000
max      45400.000000
Name: price, dtype: float64

In [ ]:
media = df.price.mean()

In [ ]:
df.loc[0].loc['price']

13495.0

In [ ]:
# definicion de la funcion a pasar
# al metodo apply
def categoria(row, variable, mean):
  if row.loc[variable] >=  mean:
    return 'sup'
  else:
    return 'inf'


df.apply(categoria, axis = 1, args =['price', media])

0      sup
1      sup
2      sup
3      sup
4      sup
      ... 
196    sup
197    sup
198    sup
199    sup
200    sup
Length: 201, dtype: object

Pasemos esta función al método indicando que queremos realizar la operación por filas:

In [ ]:
df['categ_price'] = df.apply(categoria,
                             axis = 1,
                             args =['price',
                                    media])

Modifiquemos la variable en nuestro conjunto de datos:

In [ ]:
df.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,categ_price
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1,sup
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1,sup
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,...,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1,sup
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,...,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1,sup
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,...,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1,sup


Note que en la función que se construya para pasar al método `apply`, se pueden evaluar condiciones sobre múltiples columnas para realizar una operación, aunque esta sólo actúe sobre una de ellas. Más adelante veremos ejemplos donde esto pueda ser útil.

Este tipo de operación, aunque nos sirvió para ilustrar el método, se puede realizar fácilmente con la función `cut` de Pandas, la cual nos permite segmentar una variable (*price* en este caso) en ciertos *bins*, a los cuales podemos asociar de forma opcional una etiqueta mediante el kwarg `labels`.

Por ejemplo, realicemos la conversión anterior pero ahora incluyendo un tercer valor adicional `med`:

In [ ]:
pd.cut(df.price, bins = 3, labels = ['inf', 'med', 'sup'])

0      inf
1      inf
2      inf
3      inf
4      inf
      ... 
196    inf
197    med
198    med
199    med
200    med
Name: price, Length: 201, dtype: category
Categories (3, object): ['inf' < 'med' < 'sup']

In [ ]:
p

Modifiquemos la variable en nuestro conjunto de datos:

In [ ]:
df.categ_price = pd.cut(df.price, bins = 3, labels = ['inf', 'med', 'sup'])
df.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,categ_price
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1,inf
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1,inf
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,...,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1,inf
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,...,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1,inf
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,...,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1,inf


<p><a name="gro"></a></p>

## **GroupBy: dividir, aplicar y combinar**

Las agregaciones simples pueden darnos una idea del conjunto de datos, pero a menudo preferimos agregar condicionalmente sobre alguna etiqueta o índice: esto se implementa en la llamada operación *groupby*

El objeto GroupBy es una abstracción muy flexible. En muchos sentidos, podemos tratarlo como si fuera una colección de DataFrames, para los cuales se realizan una serie de operaciones complejas "por debajo".

En la siguiente figura se ilustra un ejemplo canónico de esta operación dividir-aplicar-combinar, donde "aplicar" hace referencia a la función de agregación *apply*:

![groupby](https://lewtun.github.io/dslectures/images/split-apply-combine.png)

* El paso de división implica dividir y agrupar un DataFrame según el valor de la clave especificada.
* El paso de aplicación implica calcular alguna función, generalmente una agregación, dentro de los grupos individuales.
* El paso de combinación fusiona los resultados de estas operaciones en un objeto (Serie o DataFrame) como salida.

Si bien esto ciertamente se podría hacer manualmente usando alguna combinación de las funcionalidades de enmascaramiento, agregación y fusión descritos anteriormente, lo importante es que las divisiones intermedias no necesitan ser instanciadas explícitamente. El poder de GroupBy es que abstrae estos pasos: el usuario no necesita pensar en cómo se realiza el cálculo "por debajo", sino que piensa en la operación como un todo.

Veamos un ejemplo concreto con el conjunto de datos que hemos cargado



In [ ]:
df.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,categ_price
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1,inf
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1,inf
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,...,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1,inf
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,...,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1,inf
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,...,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1,inf


Elijamos como clave de agrupación la variable `make`:

In [ ]:
df.groupby('make')

Note que lo que se devuelve no es un conjunto de DataFrames, sino un objeto `DataFrameGroupBy`. Para producir un resultado, podemos aplicar un agregado a este objeto, que realizará los pasos apropiados de aplicación / combinación para producir el resultado deseado. Por ejemplo apliquemos la función de agregación `count`:


In [ ]:
df.groupby('make').count().loc['jaguar']

symboling            3
normalized-losses    3
aspiration           3
num-of-doors         3
body-style           3
drive-wheels         3
engine-location      3
wheel-base           3
length               3
width                3
height               3
curb-weight          3
engine-type          3
num-of-cylinders     3
engine-size          3
fuel-system          3
bore                 3
stroke               3
compression-ratio    3
horsepower           3
peak-rpm             3
city-mpg             3
highway-mpg          3
price                3
city-L/100km         3
horsepower-binned    2
diesel               3
gas                  3
categ_price          3
Name: jaguar, dtype: int64

Note que obtenemos como salida cuántos elementos, por constructora, aparecen en cada columna. Obviamente todas las columnas tienen el mismo valor, por lo que podemos seleccionar alguna columna que queramos visualizar

In [ ]:
df.groupby('make').symboling.count()

make
alfa-romero       3
audi              6
bmw               8
chevrolet         3
dodge             9
honda            13
isuzu             2
jaguar            3
mazda            17
mercedes-benz     8
mercury           1
mitsubishi       13
nissan           18
peugot           11
plymouth          7
porsche           4
renault           2
saab              6
subaru           12
toyota           32
volkswagen       12
volvo            11
Name: symboling, dtype: int64

In [ ]:
df.groupby('make')['symboling'].count()

make
alfa-romero       3
audi              6
bmw               8
chevrolet         3
dodge             9
honda            13
isuzu             2
jaguar            3
mazda            17
mercedes-benz     8
mercury           1
mitsubishi       13
nissan           18
peugot           11
plymouth          7
porsche           4
renault           2
saab              6
subaru           12
toyota           32
volkswagen       12
volvo            11
Name: symboling, dtype: int64

Alternativamente podemos utilizar el método `size`:

In [ ]:
df.groupby('make').size()

make
alfa-romero       3
audi              6
bmw               8
chevrolet         3
dodge             9
honda            13
isuzu             2
jaguar            3
mazda            17
mercedes-benz     8
mercury           1
mitsubishi       13
nissan           18
peugot           11
plymouth          7
porsche           4
renault           2
saab              6
subaru           12
toyota           32
volkswagen       12
volvo            11
dtype: int64

Si ordenamos esta salida en forma descendente obtendremos la misma salida de la aplicación del método `value_counts` sobre la serie `df.make`:

In [ ]:
df.make.value_counts()

toyota           32
nissan           18
mazda            17
mitsubishi       13
honda            13
volkswagen       12
subaru           12
peugot           11
volvo            11
dodge             9
mercedes-benz     8
bmw               8
plymouth          7
audi              6
saab              6
porsche           4
jaguar            3
chevrolet         3
alfa-romero       3
isuzu             2
renault           2
mercury           1
Name: make, dtype: int64

In [ ]:
df.groupby('make').size().sort_values(ascending= False)

make
toyota           32
nissan           18
mazda            17
mitsubishi       13
honda            13
volkswagen       12
subaru           12
peugot           11
volvo            11
dodge             9
mercedes-benz     8
bmw               8
plymouth          7
audi              6
saab              6
porsche           4
jaguar            3
chevrolet         3
alfa-romero       3
isuzu             2
renault           2
mercury           1
dtype: int64

En este caso obviamente la mejor opción es utilizar el método `value_counts`, lo importante es ver la libertad que nos proporciona el `groupby` para agrupar y realizar operaciones sobre los datos.

Veamos algunos otros ejemplos:

In [ ]:
df.groupby('make').price.max()

make
alfa-romero      16500.0
audi             23875.0
bmw              41315.0
chevrolet         6575.0
dodge            12964.0
honda            12945.0
isuzu            11048.0
jaguar           36000.0
mazda            18344.0
mercedes-benz    45400.0
mercury          16503.0
mitsubishi       14869.0
nissan           19699.0
peugot           18150.0
plymouth         12764.0
porsche          37028.0
renault           9895.0
saab             18620.0
subaru           11694.0
toyota           17669.0
volkswagen       13845.0
volvo            22625.0
Name: price, dtype: float64

In [ ]:
# agrupar por constructora y obtener el minimo y maximo precio
# ordenando la salida por la columna "min"
df.groupby('make').price.agg(['max', np.min]).sort_values(by = 'amin')

,max,amin
make,,
subaru,11694.0,5118.0
chevrolet,6575.0,5151.0
mazda,18344.0,5195.0
toyota,17669.0,5348.0
mitsubishi,14869.0,5389.0
honda,12945.0,5399.0
nissan,19699.0,5499.0
dodge,12964.0,5572.0
plymouth,12764.0,5572.0


Incluyamos ahora otra función que nos mida la diferencia entre el precio máximo y el mínimo, y ordenemos de forma descendente respecto a esta columna:

In [ ]:
def difference(s):
  return s.max() - s.min()

df.groupby('make').price.agg(['max', np.min, difference])\
  .sort_values(by = 'difference', ascending = False)

,max,amin,difference
make,,,
bmw,41315.0,16430.0,24885.0
mercedes-benz,45400.0,25552.0,19848.0
porsche,37028.0,22018.0,15010.0
nissan,19699.0,5499.0,14200.0
mazda,18344.0,5195.0,13149.0
toyota,17669.0,5348.0,12321.0
audi,23875.0,13950.0,9925.0
volvo,22625.0,12940.0,9685.0
mitsubishi,14869.0,5389.0,9480.0


Ahora, *groupby* nos permite agrupar por más de una clave. Agrupemos ahora por constructura y gasto de combustible en ciudad "city-mpg" *(miles per gallon)* para obtener los valores mínimos, máximos y su respectiva diferencia para los precios:

In [ ]:
df.groupby(['make', 'city-mpg']).price.agg(['min', 'max', difference])

min      max  difference
make        city-mpg                              
alfa-romero 19        16500.0  16500.0         0.0
            21        13495.0  16500.0      3005.0
audi        17        23875.0  23875.0         0.0
            18        17450.0  17450.0         0.0
            19        15250.0  18920.0      3670.0
...                       ...      ...         ...
volvo       18        21485.0  21485.0         0.0
            19        19045.0  22625.0      3580.0
            23        12940.0  16845.0      3905.0
            24        15985.0  16515.0       530.0
            26        22470.0  22470.0         0.0

[94 rows x 3 columns]

Note que en este caso hemos obtenido un DataFrame que contiene no un índice sino dos. Más adelante estudiaremos este tipo de estructuras.